In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.facecolor'] = 'w'
# from scipy import stats
# from scipy.special import beta as beta_function

In [19]:
# T = np.array([[0.2, 0.6, 0.2], 
#               [0.3, 0.0, 0.7],
#               [0.5, 0.0, 0.5]]).T
T = np.array([[0.2, 0.3, 0.5], 
              [0.6, 0.0, 0.0],
              [0.2, 0.7, 0.5]])

x0 = np.array([[0], 
               [1],
               [0]])

In [20]:
np.dot(T,x0)

array([[0.3],
       [0. ],
       [0.7]])

In [21]:
np.dot(T, np.dot(T,x0) )

array([[0.41],
       [0.18],
       [0.41]])

In [22]:
xn = x0
for _ in range(15):
    print(xn.T)
    xn = np.round(np.dot(T, xn), 2)

[[0 1 0]]
[[0.3 0.  0.7]]
[[0.41 0.18 0.41]]
[[0.34 0.25 0.41]]
[[0.35 0.2  0.45]]
[[0.36 0.21 0.43]]
[[0.35 0.22 0.43]]
[[0.35 0.21 0.44]]
[[0.35 0.21 0.44]]
[[0.35 0.21 0.44]]
[[0.35 0.21 0.44]]
[[0.35 0.21 0.44]]
[[0.35 0.21 0.44]]
[[0.35 0.21 0.44]]
[[0.35 0.21 0.44]]
